# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Machine Learning e IA em Ambientes Distribuídos</font>

# Projeto - Parte 3 - Etiquetagem, Seleção de Atributos e Modelagem

In [ ]:
# Imports
import sys
import os
import re
import time
import datetime
import atexit
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyspark.sql.functions as F

from pyspark import SparkConf
from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import col,udf,lag,date_add,explode,lit,concat,unix_timestamp,sum, abs
from pandas import DataFrame
from pyspark.sql.dataframe import *
from pyspark.ml.classification import *
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.window import Window
from pyspark.sql.types import DateType
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.sql import Row
from pyspark.ml import Pipeline, PipelineModel
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, RFormula
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from sklearn.metrics import roc_curve,auc
from pyspark.sql.functions import month, weekofyear, dayofmonth
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import PCA
from pyspark.ml.feature import MinMaxScaler
from pyspark.sql.types import DoubleType
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.grid_search import ParameterGrid

## Carregar Resultado do Notebook #2 

In [2]:
# Carregando resultados do Notebook #2 
df = sqlContext.read.parquet('notebook2_resultado.parquet')

# Verifique a dimensão do conjunto e garanta que esteja tudo ok
print(df.count(), len(df.columns))
df.select('key','deviceid').show(3)

(2085102, 1277)
+--------------------+--------+
|                 key|deviceid|
+--------------------+--------+
|N0001_2012-12-08 ...|   N0001|
|N0001_2012-12-09 ...|   N0001|
|N0001_2012-12-10 ...|   N0001|
+--------------------+--------+
only showing top 3 rows



## Construção de Etiquetas (Labels)

Para casos de uso de manutenção preditiva, geralmente queremos prever a falha / problema antecipadamente. Em nosso exemplo, gostaríamos de prever problemas nas máquinas com 7 dias de antecedência (janela de tempo de previsão de falha). Isso significa que, para a coluna de rótulos, precisamos rotular todos os 7 dias antes do dia de falha / problema real, como "1". Essa janela de tempo deve ser escolhida com base no caso de negócios específico: em algumas situações, pode ser suficiente prever antecipadamente as falhas com horas, enquanto em outros casos dias ou até semanas podem ser necessárias para tomar decisões comerciais significativas, como permitir tempo suficiente para a chegada de peças de reposição.

Para encontrar informações mais detalhadas sobre a técnica de construção de etiquetas, visite este link:(https://pdfs.semanticscholar.org/284d/f4ec85eed338a87fece985246c5bd4f56495.pdf).


In [3]:
#------------------------------------ Criando label column ------------------------------------#

# Passo 1: 
df = df.withColumn('label_tmp', col('problemreported')) 

# Passo 2:
wSpec = Window.partitionBy('deviceid').orderBy(df.date.desc())
lag_window = 7 

for i in range(lag_window):
    lag_values = lag(df.label_tmp, default=0).over(wSpec)
    df = df.withColumn('label_tmp', F.when((col('label_tmp')==1) | (lag_values==None) | (lag_values<1) | (lag_values>=(lag_window+1)), col('label_tmp')).otherwise(lag_values+1))

# Verificando resultados
print(df.select('label_tmp').distinct().rdd.map(lambda r: r[0]).collect()) 

[1, 6, 3, 5, 4, 8, 7, 2, 0]


In [4]:
# Passo 3:
# Note que precisamos fazer o "rótulo" como double em vez de inteiro para os modelos de classificação pyspark
df = df.withColumn('label', F.when(col('label_tmp') > 0, 1.0).otherwise(0.0))
df.createOrReplaceTempView("df_view") 
 
# Passo 4:
df.orderBy('deviceid', 'date').select('deviceid', 'date', 'problemreported', 'label_tmp', 'label').show(20) 

+--------+--------------------+---------------+---------+-----+
|deviceid|                date|problemreported|label_tmp|label|
+--------+--------------------+---------------+---------+-----+
|   N0001|2012-12-08 00:00:...|              0|        0|  0.0|
|   N0001|2012-12-09 00:00:...|              0|        0|  0.0|
|   N0001|2012-12-10 00:00:...|              0|        0|  0.0|
|   N0001|2012-12-11 00:00:...|              0|        0|  0.0|
|   N0001|2012-12-12 00:00:...|              0|        0|  0.0|
|   N0001|2012-12-13 00:00:...|              0|        0|  0.0|
|   N0001|2012-12-14 00:00:...|              0|        0|  0.0|
|   N0001|2012-12-15 00:00:...|              0|        0|  0.0|
|   N0001|2012-12-16 00:00:...|              0|        0|  0.0|
|   N0001|2012-12-17 00:00:...|              0|        8|  1.0|
|   N0001|2012-12-18 00:00:...|              0|        7|  1.0|
|   N0001|2012-12-19 00:00:...|              0|        6|  1.0|
|   N0001|2012-12-20 00:00:...|         

In [5]:
# Visualize a distribuição da coluna "label"
df.select('label').describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|            2085102|
|   mean|0.01456187754843648|
| stddev|0.11979080162007248|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



## Redução de Dimensionalidade
- Não há muitos pacotes para seleção de recursos no PySpark, Por isso decidimos usar o PCA para reduzir a densidade.
- Precisamos realizar a seleção de recursos para reduzir o tamanho do conjunto de recursos

In [6]:
# Verificando o número de rolling features
len([col_n for col_n in df.columns if '_rolling' in col_n])

1150

In [7]:
# Passo 1
# Usando RFormula para criar um vetor de atributos
rolling_features = list(s for s in df.columns if "_rolling" in s)
formula = RFormula(formula = "~" + "+".join(rolling_features))
output = formula.fit(df).transform(df).select("key","features") 


# Passo 2 
# Antes do PCA, precisamos padronizar os recursos, é muito importante ...
# Nós comparamos 1) padronização, 2) min-max normalização, 3) combinação de padronização e min-max normalização

# Em 2), o 1º PC explicou mais de 67% da variância
# 1) & 3) geram exatamente os mesmos resultados para model.explainedVariance.
# Isso significa que a normalização min-max não ajuda em nosso caso

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

# Compute estatísticas de resumo ajustando o StandardScaler
scalerModel = scaler.fit(output)

# Normalize cada recurso para ter desvio padrão de 1.
scaledData = scalerModel.transform(output)


# Passo 3
pca = PCA(k=50, inputCol="scaledFeatures", outputCol="pca_roll_features")
model = pca.fit(scaledData)
result = model.transform(scaledData).select("key","pca_roll_features")
print(model.explainedVariance)


# Passo 4
df = df.join(result, 'key', 'inner')
rolling_drop_list = [col_n for col_n in df.columns if '_rolling' in col_n]
df = df.select([column for column in df.columns if column not in rolling_drop_list])

df.select('key','pca_roll_features').show(5)


[0.0720783785227,0.0329825870736,0.027901485354,0.0243662075346,0.0231109612038,0.0230696286064,0.0175345239922,0.0169050432163,0.0160163382589,0.0151685106261,0.0139611893027,0.0132999118492,0.0128137208726,0.012460010309,0.0123226258655,0.0121766225655,0.0110137480593,0.0106161694018,0.0103270115698,0.00977105423534,0.00908164998253,0.00880547728788,0.00839688758961,0.00833527942731,0.00816512952224,0.00807362710216,0.00787447043078,0.00782728526041,0.00776590544977,0.00753956685426,0.00735611311084,0.00717316781783,0.00710561994143,0.00694768452248,0.0068423949855,0.00669806943767,0.00660469671941,0.00658491264526,0.00646747772504,0.00638036477672,0.00630822833773,0.00628238685298,0.00608372864476,0.00606093314036,0.00604486211127,0.00599575013941,0.00591032102729,0.0057055133175,0.00568659059503,0.00551330593664]
+--------------------+--------------------+
|                 key|   pca_roll_features|
+--------------------+--------------------+
|N0001_2012-12-08 ...|[0.0,0.0,0.0,0.0,

# Preparando os Dados de Treino e Teste

In [8]:
# Definir lista de colunas de entrada para modelagem 
input_features = [
 'usage_count_1',
 'usage_count_2',
 'problem_type_1',
 'problem_type_2',
 'problem_type_3',
 'problem_type_4',
 'error_count_1',
 'error_count_2',
 'error_count_3',
 'error_count_4',
 'error_count_5',
 'error_count_6',
 'error_count_7',
 'error_count_8',
 'month',
 'weekofyear',
 'dayofmonth',
 'warn_type1_total',
 'warn_type2_total',
 'fault_code_type_1_count',
 'fault_code_type_2_count',
 'fault_code_type_3_count',
 'fault_code_type_4_count',
 'problem_type_1_per_usage1',
 'problem_type_2_per_usage1',
 'problem_type_3_per_usage1',
 'problem_type_4_per_usage1',
 'fault_code_type_1_count_per_usage1',
 'fault_code_type_2_count_per_usage1',
 'fault_code_type_3_count_per_usage1',
 'fault_code_type_4_count_per_usage1',
 'problem_type_1_per_usage2',
 'problem_type_2_per_usage2',
 'problem_type_3_per_usage2',
 'problem_type_4_per_usage2',
 'fault_code_type_1_count_per_usage2',
 'fault_code_type_2_count_per_usage2',
 'fault_code_type_3_count_per_usage2',
 'fault_code_type_4_count_per_usage2',   
 'problem_type_1_category_encoded',
 'problem_type_2_category_encoded',
 'problem_type_3_category_encoded',
 'problem_type_4_category_encoded',
 'problem_type_1_per_usage1_category_encoded',
 'problem_type_2_per_usage1_category_encoded',
 'problem_type_3_per_usage1_category_encoded',
 'problem_type_4_per_usage1_category_encoded',
 'problem_type_1_per_usage2_category_encoded',
 'problem_type_2_per_usage2_category_encoded',
 'problem_type_3_per_usage2_category_encoded',
 'problem_type_4_per_usage2_category_encoded',
 'fault_code_type_1_count_category_encoded',
 'fault_code_type_2_count_category_encoded',
 'fault_code_type_3_count_category_encoded',
 'fault_code_type_4_count_category_encoded',
 'fault_code_type_1_count_per_usage1_category_encoded',
 'fault_code_type_2_count_per_usage1_category_encoded',
 'fault_code_type_3_count_per_usage1_category_encoded',
 'fault_code_type_4_count_per_usage1_category_encoded',
 'fault_code_type_1_count_per_usage2_category_encoded',
 'fault_code_type_2_count_per_usage2_category_encoded',
 'fault_code_type_3_count_per_usage2_category_encoded',
 'fault_code_type_4_count_per_usage2_category_encoded',
 'cat1_encoded',
 'cat2_encoded',
 'cat3_encoded',
 'cat4_encoded',     
 'pca_1_warn',
 'pca_2_warn',
 'pca_3_warn',
 'pca_4_warn',
 'pca_5_warn',
 'pca_6_warn',
 'pca_7_warn',
 'pca_8_warn',
 'pca_9_warn',
 'pca_10_warn',
 'pca_11_warn',
 'pca_12_warn',
 'pca_13_warn',
 'pca_14_warn',
 'pca_15_warn',
 'pca_16_warn',
 'pca_17_warn',
 'pca_18_warn',
 'pca_19_warn',
 'pca_20_warn',
 'pca_roll_features'
]

label_var = ['label']
key_cols =['key','deviceid','date']

In [9]:
# Juntando as features
va = VectorAssembler(inputCols=(input_features), outputCol='features')
df = va.transform(df).select('deviceid','date','label','features')

In [10]:
# Configure maxCategories para que os recursos com > 10 valores distintos sejam tratados como contínuos.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(df)

#### Lembre-se de fazer o "StringIndexer" na coluna de rótulo e ajustar em todo o conjunto de dados para incluir todos os rótulos no índice. Além disso, a coluna de rótulo deve ser Double em vez do tipo Integer.

In [11]:
# Ajuste todo o conjunto de dados para incluir todos os rótulos no índice.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)

### Prepare o conjunto de dados de treino e de teste usando o método de divisão de tempo
- dados de formação: ano 2012-2014
- dados de teste: ano de 2015

In [12]:
training = df.filter(df.date > "2011-12-31").filter(df.date < "2015-01-01")
testing = df.filter(df.date > "2014-12-31")

print(training.count())
print(testing.count())


1431846
655155


In [13]:
# Mostra a distribuição do rótulo "0" e "1"
df.groupby('label').count().show()

+-----+-------+
|label|  count|
+-----+-------+
|  0.0|2054739|
|  1.0|  30363|
+-----+-------+



### Exemplos negativos de amostra negativa:

- Este é um *** altamente desequilibrado conjunto de dados *** com muito mais rótulo "0" do que "1" ("1" representa apenas 1,5%).
- Então, precisamos balancear os dados.
- Para tornar a etiqueta "1" em "0" próxima de 1:10 (você pode usar outra razão, por exemplo, 1:5), precisamos testar os "0" s (obter 13,5% de todos os rótulos " 0 "s)

In [14]:
# SampleBy retorna uma amostra estratificada sem reposição com base na fração dada em cada estrato
train_downsampled = training.sampleBy('label', fractions={0.0: 0.135, 1.0: 1.0}, seed=123).cache()
train_downsampled.groupby('label').count().show()

testing.groupby('label').count().show()

+-----+------+
|label| count|
+-----+------+
|  0.0|190823|
|  1.0| 20777|
+-----+------+

+-----+------+
|label| count|
+-----+------+
|  0.0|645547|
|  1.0|  9608|
+-----+------+



### Resultados do cache

Faça isso quando necessário, especialmente se o seu trabalho (por exemplo, modelagem recursiva) usar esses dados repetidas vezes. Aqui no nosso caso, depois que os conjuntos de dados de treino e teste são preparados, nós os armazenamos em cache na memória.

In [15]:
# Cache datasets in memory
train_downsampled.cache()
testing.cache()

# Verifique o número de dispositivos em treinamento e teste
print(train_downsampled.select('deviceid').distinct().count())
print(testing.select('deviceid').distinct().count())


1899
1899


In [16]:
# Definir o caminho do diretório de armazenamento do modelo. É aqui que os modelos serão salvos.
modelDir = "/Outputs/"; 

## Modelos de Classificação Binária:
- Classificador Random Forest
- Árvore impulsionada por gradiente (Tree Gradient-Boosted)

### Classificador Random Forest 

In [17]:
%%time

# Criando um modelo RandomForest.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=100)

# Indexadores e floresta em um pipeline
pipeline_rf = Pipeline(stages=[labelIndexer, featureIndexer, rf])

# Treinando o modelo
model_rf = pipeline_rf.fit(train_downsampled)

# Salvando o modelo
datestamp = unicode(datetime.datetime.now()).replace(' ','').replace(':','_');
rf_fileName = "RandomForest_" + datestamp;
rfDirfilename = modelDir + rf_fileName;
model_rf.save(rfDirfilename)

# Fazendo perevisões
predictions_rf = model_rf.transform(testing)
predictions_rf.groupby('indexedLabel', 'prediction').count().show()

+------------+----------+------+
|indexedLabel|prediction| count|
+------------+----------+------+
|         1.0|       1.0|   478|
|         0.0|       1.0|  1746|
|         1.0|       0.0|  9130|
|         0.0|       0.0|643801|
+------------+----------+------+

CPU times: user 20.5 ms, sys: 13.6 ms, total: 34.1 ms
Wall time: 28.8 s


In [18]:
predictions_rf.dtypes

[('deviceid', 'string'),
 ('date', 'timestamp'),
 ('label', 'double'),
 ('features', 'vector'),
 ('indexedLabel', 'double'),
 ('indexedFeatures', 'vector'),
 ('rawPrediction', 'vector'),
 ('probability', 'vector'),
 ('prediction', 'double')]

In [19]:
%%time

predictionAndLabels = predictions_rf.select("indexedLabel", "prediction").rdd
metrics = BinaryClassificationMetrics(predictionAndLabels)
print("Area under ROC = %g" % metrics.areaUnderROC)
print("Area under PR = %g\n" % metrics.areaUnderPR)

# Select (prediction, true label)e calcula o erro em teste
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction")
print("Accuracy = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "accuracy"}))
print("Weighted Precision = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "weightedPrecision"}))
print("Weighted Recall = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "weightedRecall"}))
print("F1 = %g" % evaluator.evaluate(predictions_rf, {evaluator.metricName: "f1"}))

# PLOT ROC curve após converter predictions para um dataframe do Pandas
%matplotlib inline
predictions_rf_pddf = predictions_rf.select('indexedLabel','probability').toPandas()
labels = predictions_rf_pddf["indexedLabel"]
prob = []
for dv in predictions_rf_pddf["probability"]:
    prob.append(dv.values[1])
     
fpr, tpr, thresholds = roc_curve(labels, prob, pos_label=1.0);
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()


Area under ROC = 0.600472
Area under PR = 0.133672

Accuracy = 0.983399
Weighted Precision = 0.974709
Weighted Recall = 0.983399
F1 = 0.978267


CPU times: user 1min 22s, sys: 786 ms, total: 1min 23s
Wall time: 2min 14s


#### O Pyspark MulticlassClassificationEvaluator apenas fornece precisão ponderada e recuperação. Mas também gostaríamos de ver a precisão e o recall.

In [20]:
# Use sklearn
rf_result = predictions_rf.select('indexedLabel', 'prediction').toPandas()

rf_label = rf_result['indexedLabel'].tolist()
rf_prediction = rf_result['prediction'].tolist()

precision, recall, fscore, support = score(rf_label, rf_prediction)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))


precision: [ 0.9860169   0.21492806]
recall: [ 0.99729532  0.04975021]
fscore: [ 0.99162404  0.08079784]
support: [645547   9608]


### Classificador Tree Gradient-Boosted 

In [21]:
%%time

# Cria o modelo
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxDepth=10, minInstancesPerNode=5, maxIter=50)

# Index e modelo no Pipeline
pipeline_gbt = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Treina o modelo
model_gbt = pipeline_gbt.fit(train_downsampled)

# Salva o modelo
datestamp = unicode(datetime.datetime.now()).replace(' ','').replace(':','_');
gbt_fileName = "GradientBoostedTree_" + datestamp;
gbtDirfilename = modelDir + gbt_fileName;
model_gbt.save(gbtDirfilename)

# Faz previsões
predictions_gbt = model_gbt.transform(testing)

CPU times: user 127 ms, sys: 13.1 ms, total: 140 ms
Wall time: 12min 9s


In [22]:
# Mostra as previsões
predictions_gbt.groupby('indexedLabel', 'prediction').count().show()

+------------+----------+------+
|indexedLabel|prediction| count|
+------------+----------+------+
|         1.0|       1.0|   831|
|         0.0|       1.0|  6332|
|         1.0|       0.0|  8777|
|         0.0|       0.0|639215|
+------------+----------+------+



In [23]:
%%time

predictionAndLabels = predictions_gbt.select("indexedLabel", "prediction").rdd
metrics = BinaryClassificationMetrics(predictionAndLabels)
print("Area under ROC = %g" % metrics.areaUnderROC)
print("Area under PR = %g\n" % metrics.areaUnderPR)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction")
print("Accuracy = %g" % evaluator.evaluate(predictions_gbt, {evaluator.metricName: "accuracy"}))
print("Weighted Precision = %g" % evaluator.evaluate(predictions_gbt, {evaluator.metricName: "weightedPrecision"}))
print("Weighted Recall = %g" % evaluator.evaluate(predictions_gbt, {evaluator.metricName: "weightedRecall"}))
print("F1 = %g" % evaluator.evaluate(predictions_gbt, {evaluator.metricName: "f1"}))

Area under ROC = 0.551234
Area under PR = 0.106084

Accuracy = 0.976938
Weighted Precision = 0.97369
Weighted Recall = 0.976938
F1 = 0.975279
CPU times: user 54.4 ms, sys: 20.3 ms, total: 74.7 ms
Wall time: 40.6 s


In [24]:
# Usando sklearn para calcular precision e recall

gbt_result = predictions_gbt.select('indexedLabel', 'prediction').toPandas()

gbt_label = gbt_result['indexedLabel'].tolist()
gbt_prediction = gbt_result['prediction'].tolist()

precision, recall, fscore, support = score(gbt_label, gbt_prediction)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))


precision: [ 0.98645508  0.11601284]
recall: [ 0.99019126  0.08649042]
fscore: [ 0.98831964  0.09909964]
support: [645547   9608]


### Comparando os resultados de Random Forest e Gradient Boosted Tree:
- Gradient Boosted Tree tem um melhor recall, mas pior precisão em comparação com a Random Forest. Para a maioria dos casos de uso de manutenção preditiva, o custo de negócios associado a falsos positivos geralmente é caro. Há sempre um trade-off entre precisão e recall. Queremos obter uma taxa de precisão mais alta (menos falsos positivos), mesmo que isso possa comprometer a taxa de recall.
- É por isso que decidimos ir com o modelo Random Forest e otimizamos ainda mais com o ajuste de hiperparametros.

## Hyper-Parameter Tuning & Cross Validation

Treinar um modelo de classificaçãousando ajuste de hiperparâmetros e validação cruzada.

In [25]:
%%time

# Cria o modelo
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", 
                            featureSubsetStrategy="auto", impurity="gini", seed=123)

# Index e modelo no Pipeline
pipeline_rf = Pipeline(stages=[labelIndexer, featureIndexer, rf])


# Define os parâmetros de grid
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [20, 50, 100]) \
    .addGrid(rf.maxBins, [10, 20]) \
    .addGrid(rf.maxDepth, [3, 5, 7]) \
    .addGrid(rf.minInstancesPerNode, [1, 5, 10]) \
    .build()

# Cross-validation
crossval = CrossValidator(estimator=pipeline_rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName="weightedPrecision"),
                          numFolds=3)

# Treinamento usando CV
cvModel = crossval.fit(train_downsampled)

# Previsões
predictions = cvModel.transform(testing)
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="weightedPrecision")
r2 = evaluator.evaluate(predictions)
print("weightedPrecision on test data = %g" % r2)

# Salvando o melhor modelo
fileName = "CV_RandomForestClassificationModel_" + datestamp;
CVDirfilename = modelDir + fileName;
cvModel.bestModel.save(CVDirfilename);

weightedPrecision on test data = 0.97472
CPU times: user 3.4 s, sys: 1.8 s, total: 5.2 s
Wall time: 54min 1s


# Fim